# AMORE datasets

- Note: Years 1997 to 1999 contain only a few reviews and are not included in Doc2Vec embeddings.  
  (The years 2000 to 2012 are included.)

In [1]:
# Reload modules every time before executing the Python code typed
%load_ext autoreload
%autoreload 2

# Import from project root
import sys; sys.path.insert(0, '../')

import bz2
import pickle
import timeit
import numpy as np
from scipy.sparse import csr_matrix

from access.file_storage import FileStorage
from access.interim_storage import InterimStorage
from amore.amazon_reviews_reader import AmazonReviewsReader

## Read data

* **opinion_lexicon**: Instance of OpinionLexicon to access negative and positive words. Not used for now.
* **year_star_ids**: Collection of IDs sorted by years and stars. Used afterwards to filter by stars and years in method **get_review_ids**.
* **reader**: Instance of AmazonReviewsReader to access review data. Used afterwards to create revno_to_text.
* **revno_to_text**: Dictionary review-number to full text. Used afterwards to access review-texts in method **get_text**.

In [2]:
# For multiple usage afterwards
file_storage = FileStorage()

# Already used before, not required in this version.
if False:
    from amore.opinion_lexicon import OpinionLexicon
    opinion_lexicon = OpinionLexicon(file_storage.get_filepath('opinion-words'))
    print('negative words:', len(opinion_lexicon.get_negative_set()))
    print('positive words:', len(opinion_lexicon.get_positive_set()))
    # negative words: 4783
    # positive words: 2006

In [3]:
# Read deduplicated review Ids
with bz2.BZ2File(file_storage.get_filepath('deduplicated'), 'r') as file:
    year_star_ids = pickle.loads(file.read())
print('Available years:', sorted(year_star_ids.keys()))
print('Example stars:  ', sorted(year_star_ids[2007].keys()))
print('Example entry:  ', year_star_ids[2007][1][0])
count = 0
for year in year_star_ids:
    for star in year_star_ids[year]:
        count += len(year_star_ids[year][star])
print('Reviews:', count)

# Available years: Available years: [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]
# Example stars:   [1, 2, 3, 4, 5]
# Example entry:   [4368, 2007, 1]
# Reviews: 1727821

Available years: [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]
Example stars:   [1, 2, 3, 4, 5]
Example entry:   [4368, 2007, 1]
Reviews: 1727821


In [4]:
# Read review texts

# Configuration: Only texts of reviews to be included afterwards have be be read.
min_year = 2006
max_year = 2008

max_docs = -1
start_time = timeit.default_timer()
reader = AmazonReviewsReader(file_storage.get_filepath('amazon_gz_file'), AmazonReviewsReader.MODE_TYPED, min_year=min_year, max_year=max_year, max_docs=max_docs)
revno_to_text = {}
def get_texts(item):
    return (item[AmazonReviewsReader.KEY_SUMMARY] + " " + item[AmazonReviewsReader.KEY_TEXT]).replace('<br />', ' ')
for item in reader:
    revno_to_text[item[AmazonReviewsReader.KEY_NUMBER]] = get_texts(item)
print('Texts:', len(revno_to_text))
print('Runtime:', timeit.default_timer() - start_time)

# start year: 2007
# Texts: 4662381
# Runtime: 265.2943881880492

# start year: 2000
# Texts: 7827594
# Runtime: 312.4320105519146

Texts: 2129442
Runtime: 253.98197159497067


### Read document-term matrix files

Data required to create **Matrix** instances afterwards.

* **doc_term_matrix**: Matrix containing document-IDs and term-IDs. Used afterwards for building Matrix.
* **vocabulary**: Dictionary containing tuples of terms and term-IDs. Used afterwards to create inv_vocabulary.
* **inv_vocabulary**: Dictionary containing tuples of term-IDs and terms.Used afterwards to create tokens.
* **tokens**: Ordered list of terms, index represents term-ID. Used afterwards as token-indices for building Matrix.
* **vecid_revno**: Values used afterwards as review-indices for building Matrix.

In [5]:
# Read document-term matrix
start_time = timeit.default_timer()
with bz2.BZ2File(file_storage.get_filepath('AMORE-CountVec-DocTermMatrix'), 'r') as file:
    doc_term_matrix = pickle.loads(file.read())
    print('document-term matrix:', doc_term_matrix.shape, type(doc_term_matrix))
    print('Runtime:', timeit.default_timer() - start_time)
    
#print(doc_term_matrix)
#       (0, 299799)  3
#        :       :
# (1203681, 367201)  1

# start year: 2007
# document-term matrix: (1203682, 486546) <class 'scipy.sparse.csr.csr_matrix'>
# Runtime:  26.56719038821757

# start year: 2000
# document-term matrix: (1584098, 607181) <class 'scipy.sparse.csr.csr_matrix'>
# Runtime: 54.29142002761364

document-term matrix: (1584098, 607181) <class 'scipy.sparse.csr.csr_matrix'>
Runtime: 37.495712163392454


In [6]:
# Read vocabulary of document-term matrix
# Invert it
with bz2.BZ2File(file_storage.get_filepath('AMORE-CountVec-Vocabulary'), 'r') as file:
    vocabulary = pickle.loads(file.read())
    print('vocabulary:', len(vocabulary), type(vocabulary))
    print('example:', next(iter(vocabulary.items())))

inv_vocabulary = {v: k for k, v in vocabulary.items()}
print('inv_vocabulary:', len(inv_vocabulary), type(inv_vocabulary))
print('example:', next(iter(inv_vocabulary.items())))

tokens = list(dict(sorted(inv_vocabulary.items())).values())

# start year: 2007
# vocabulary: 486546 <class 'dict'>
# example: ('movie', 299799)
# inv_vocabulary: 486546 <class 'dict'>
# example: (299799, 'movie')

# start year: 2000
# vocabulary: 607181 <class 'dict'>
# example: ('movie', 371301)
# inv_vocabulary: 607181 <class 'dict'>
# example: (371301, 'movie')

vocabulary: 607181 <class 'dict'>
example: ('movie', 371301)
inv_vocabulary: 607181 <class 'dict'>
example: (371301, 'movie')


In [7]:
# Read count-vector-ID to review-ID mapping
with bz2.BZ2File(file_storage.get_filepath('AMORE-CountVec-VecidRevno'), 'r') as file:
    vecid_revno = pickle.loads(file.read())
    print('vectorizer ID to review no:', len(vecid_revno), type(vecid_revno))
    print('example:', next(iter(vecid_revno.items())))

# start year: 2007
# vectorizer ID to review no: 1203682 <class 'dict'>
# example: (0, 3)

# start year: 2000
# vectorizer ID to review no: 1584098 <class 'dict'>
# example: (0, 3)

vectorizer ID to review no: 1584098 <class 'dict'>
example: (0, 3)


## Data access methods and Matrix class

- Docs:
    - [docs.scipy.org scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)
    - [docs.scipy.org scipy.sparse.spmatrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.spmatrix.html)
    - [docs.scipy.org sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)

In [8]:
def get_text(review_id):
    return revno_to_text[review_id]

def get_review_ids(years, stars):
    ids = []
    for year in year_star_ids:
        if year in years:
            for star in year_star_ids[year]:
                if star in stars:
                    for tup in year_star_ids[year][star]:
                        ids.append(tup[0])
    return ids

In [9]:
# Used afterwards in Matrix class.

def delete_from_csr(mat, row_indices=[], col_indices=[]):
    """
    Remove the rows (denoted by ``row_indices``) and columns (denoted by ``col_indices``) from the CSR sparse matrix ``mat``.
    WARNING: Indices of altered axes are reset in the returned matrix

    import numpy as np
    from scipy.sparse import csr_matrix
    
    https://stackoverflow.com/a/45486349
    """
    if not isinstance(mat, csr_matrix):
        raise ValueError("works only for CSR format -- use .tocsr() first")

    rows = []
    cols = []
    if row_indices:
        rows = list(row_indices)
    if col_indices:
        cols = list(col_indices)

    if len(rows) > 0 and len(cols) > 0:
        row_mask = np.ones(mat.shape[0], dtype=bool)
        row_mask[rows] = False
        col_mask = np.ones(mat.shape[1], dtype=bool)
        col_mask[cols] = False
        return mat[row_mask][:,col_mask]
    elif len(rows) > 0:
        mask = np.ones(mat.shape[0], dtype=bool)
        mask[rows] = False
        return mat[mask]
    elif len(cols) > 0:
        mask = np.ones(mat.shape[1], dtype=bool)
        mask[cols] = False
        return mat[:,mask]
    else:
        return mat

In [10]:
class Matrix:
    """
    Sparse matrix data access and filtering.
    """

    # document-term matrix, scipy.sparse.csr.csr_matrix
    doc_term_matrix = None
    
    # matrix-document-index to review-id
    review_indices = None
    reviewids_to_reviewindices = {}
    
    # matrix-term-index to token
    token_indices = None # list of tokens
    token_to_tokenindices = {}
        
    def __init__(self, doc_term_matrix: csr_matrix, review_indices: list, token_indices: list):
        self.doc_term_matrix = doc_term_matrix
        
        self.review_indices = review_indices
        self.reviewids_to_reviewindices = {}
        for i, item in enumerate(self.review_indices):
            self.reviewids_to_reviewindices[item] = i
        
        self.token_indices = token_indices
        for i, item in enumerate(self.token_indices):
            self.token_to_tokenindices[item] = i
        
        print('Created:', self)
        
    def __repr__(self):
        return 'Matrix ' + str(self.doc_term_matrix.shape) + ', ' + str(len(self.review_indices)) + ' reviews, ' + str(len(self.token_indices)) + ' tokens'

    def get_token_data(self, review_id=None, review_index=None):
        """
        Gets sparse matrix data for review index or ID.
        """
        if review_index is None:
            review_index = self.reviewids_to_reviewindices[review_id]
        return self.doc_term_matrix[review_index].data
    
    def get_token_indices(self, review_id=None, review_index=None):
        """
        Gets sparse matrix indices of tokens for review index or ID.
        """
        if review_index is None:
            review_index = self.reviewids_to_reviewindices[review_id]
        return self.doc_term_matrix[review_index].indices

    def get_token_counts(self, review_id=None, review_index=None):
        """
        Returns dict (token, count) for review index or ID.
        """
        if review_index is None:
            review_index = self.reviewids_to_reviewindices[review_id]
        token_counts = {}
        token_data = self.get_token_data(review_index=review_index)
        for i, token_index in enumerate(self.get_token_indices(review_index=review_index)):
            token = self.token_indices[token_index]
            token_counts[token] = token_data[i]
        return dict(sorted(token_counts.items(), key=lambda item: item[1], reverse=True))
    
    def filter_remove_reviews(self, review_ids):
        """
        Removes given review IDs.
        """
        review_indices_remove = []
        for review_id in review_ids:
            review_indices_remove.append(self.reviewids_to_reviewindices[review_id])
        review_indices_new = self.review_indices.copy()
        for review_index_remove in sorted(review_indices_remove, reverse=True):
            review_indices_new.pop(review_index_remove)
        return Matrix(delete_from_csr(csr_matrix(self.doc_term_matrix), review_indices_remove, []), review_indices_new, self.token_indices.copy())
    
    def filter_keep_reviews(self, review_ids):
        """
        Keeps only given review IDs.
        """
        doc_indices_extract = []
        new_review_indices = []
        for review_id in review_ids:
            doc_indices_extract.append(self.reviewids_to_reviewindices[review_id])
            new_review_indices.append(review_id)

        # Filter matrix
        print('Filtering. Based on', len(review_ids), 'review IDs')
        new_doc_term_matrix = self.doc_term_matrix[doc_indices_extract,:]
        
        return Matrix(new_doc_term_matrix, new_review_indices, self.token_indices)

    def get_overall_token_occurences(self):
        """
        Returns tokens and their occurences (counted max 1 time) in all documents.
        """
        # Count non-zero values of token-indices
        tokenindex_occurences = {}
        for tokenindex in self.doc_term_matrix.nonzero()[1]:
            if tokenindex in tokenindex_occurences:
                tokenindex_occurences[tokenindex] += 1
            else:
                tokenindex_occurences[tokenindex] = 1
                
        # Sort by values/counts
        tokenindex_occurences = dict(sorted(tokenindex_occurences.items(), key=lambda item: item[1], reverse=True))
        
        # Token-indices to tokens
        token_occurences = {}
        for item in tokenindex_occurences.items():
            token_occurences[self.token_indices[item[0]]] = item[1]
        return token_occurences

    def filter_tokens(self, tokens):
        """
        Filters matrix by a given set of tokens (e.g. positive words).
        """
        # Collect available token-indices (required to filter matrix)
        tokenindices = []
        new_token_indices = {}
        for token in tokens:
            if token in self.token_to_tokenindices:
                tokenindices.append(self.token_to_tokenindices[token])
                new_token_indices[self.token_to_tokenindices[token]] = token
        new_token_indices = dict(sorted(new_token_indices.items(), reverse=False))
        new_token_indices = list(new_token_indices.values())
        
        # Filter matrix
        new_doc_term_matrix = self.doc_term_matrix[:,tokenindices]
                
        return Matrix(new_doc_term_matrix, self.review_indices, new_token_indices)

    def filter_reviews_by_tokens(self, tokens: list) -> dict:
        """
        Returns review IDs and occurences of given tokens (counted max 1 time) in all documents.
        """
        token_indices = []
        for token in tokens:
            token_indices.append(self.token_to_tokenindices[token])
            
        reviews_to_occurences = {}
        nonzero = self.doc_term_matrix.nonzero()
        i = -1  
        for tokenindex in nonzero[1]:
            i += 1
            if tokenindex in token_indices:
                review_index = nonzero[0][i]
                if review_index in reviews_to_occurences:
                    reviews_to_occurences[review_index] += 1
                else:
                    reviews_to_occurences[review_index] = 1
        
        reviewids_to_occurences = {}
        for item in reviews_to_occurences.items():
            reviewids_to_occurences[self.review_indices[item[0]]] = item[1]
            
        return reviewids_to_occurences
    
    def count_token_occurences(self, review_ids, tokens):
        """
        Sums up the occurencies of given tokens in given reviews.
        """
        review_to_tokenoccurences = {}
        token_indices = []
        for token in tokens:
            token_indices.append(self.token_to_tokenindices[token])
        for review_id in review_ids:
            if not review_id in review_ids:
                continue
            review_to_tokenoccurences[review_id] = 0
            for counts in self.get_token_data(review_id=review_id):
                review_to_tokenoccurences[review_id] += 1
        return review_to_tokenoccurences

## Filter

#### Get tokens which occur predominantly in only one dataset: Get ratio for each token.

* **get_ratio**
  * In: matrix-A, matrix-B
  * In: min_token_occurences (token has to occure at least x times in each of both matrixes)
  * Out: tokens sorted by ratio
  * Out: tokens sorted by inverted ratio
  * Out: tokens not reached minimum
  * Out: tokens only in A
  * Out: tokens only in B
* **filter_tokens_by_ratio**
  * In: ratio (generated by method get_ratio)
  * In: min_ratio (minimum threshold)
  * In: max_tokens (maximum number of tokens to return)
  * In: exclude_tokens (tokens not to return)

In [11]:
def get_ratio(matrix_a, matrix_b, min_token_occurences = -1):
    ratio = {}
    only_a = {}
    rare = {}
    # Pairs of tokens and occurences
    a_token_occurences = matrix_a.get_overall_token_occurences()
    b_token_occurences = matrix_b.get_overall_token_occurences()
    print('Input sizes:', len(a_token_occurences), '/', len(b_token_occurences))
    for tok_occ in a_token_occurences.items():

        # Filter/remove rare:
        # minimum is set AND
        # token has to be in both matrixes AND
        # threshold reached in A AND
        # threshod reached in B AND
        if min_token_occurences != -1 and \
           tok_occ[0] in b_token_occurences and \
           min_token_occurences > tok_occ[1] and \
           min_token_occurences > b_token_occurences[tok_occ[0]]:
            rare[tok_occ[0]] = str(tok_occ[1]) + " / " + str(b_token_occurences[tok_occ[0]])
            b_token_occurences.pop(tok_occ[0])
            continue
            
        # Only A
        if not tok_occ[0] in b_token_occurences:
            only_a[tok_occ[0]] = tok_occ[1]
            continue
        
        # Add ratio, remove from B
        ratio_a = tok_occ[1] / len(a_token_occurences)
        ratio_b = b_token_occurences.pop(tok_occ[0]) / len(b_token_occurences)
        ratio[tok_occ[0]] = round(ratio_a / ratio_b, 2)

    ratio_inverted = {}
    for item in ratio.items():
        ratio_inverted[item[0]] = round(1 / item[1], 2)

    print('Output sizes:', 'ratio', len(ratio), '; rare', len(rare), '; only a', len(only_a), '; only b', len(b_token_occurences))
    return dict(sorted(ratio.items(), key=lambda item: item[1], reverse=True)), \
    dict(sorted(ratio_inverted.items(), key=lambda item: item[1], reverse=True)), \
    rare, \
    dict(sorted(only_a.items(), key=lambda item: item[1], reverse=True)), \
    dict(sorted(b_token_occurences.items(), key=lambda item: item[1], reverse=True))

def filter_tokens_by_ratio(ratio_results, min_ratio, max_tokens, exclude_tokens):
    tokens = []
    for item in ratio_results.items():
        if item[0] in exclude_tokens:
            continue
        if item[1] >= min_ratio:
            tokens.append(item[0])
        if len(tokens) >= max_tokens:
            break
    return tokens

## Set data / create base matrixes

In [12]:
# Main matrix containing all data
matrix = Matrix(doc_term_matrix, vecid_revno.values(), tokens)

Created: Matrix (1584098, 607181), 1584098 reviews, 607181 tokens


In [13]:
# Reviews to use
ids_neg_a = get_review_ids(years=[2006,2007], stars=[1,2])
ids_neg_b = get_review_ids(years=[2008], stars=[1,2])
ids_neg = ids_neg_a + ids_neg_b

ids_pos_a = []
ids_pos_b = get_review_ids(years=[2008], stars=[4,5])
ids_pos = ids_pos_a + ids_pos_b

ids_all = ids_neg + ids_pos

In [14]:
# Sub-matrixes
m_neg = matrix.filter_keep_reviews(ids_neg)
print()
m_pos = matrix.filter_keep_reviews(ids_pos)

Filtering. Based on 57263 review IDs
Created: Matrix (57263, 607181), 57263 reviews, 607181 tokens

Filtering. Based on 141293 review IDs
Created: Matrix (141293, 607181), 141293 reviews, 607181 tokens


# Get tokens to use

Note: During exploration of the data it turned out that there are more positive words that have a large ratio and therefore can be used to determine (unique) drift.

In [15]:
ratio_a, ratio_b, rare, only_a, only_b = get_ratio(
    m_pos,
    m_neg,
    min_token_occurences=10*1000)
print()
print('only_a', list(only_a.items())[:10])
print()
print('ratio_a', list(ratio_a.items())[:50])
print()
print('ratio_b', list(ratio_b.items())[:50])
print()
print('only_b', list(only_b.items())[:10])
print()
print('rare', list(rare.items())[:10])

Input sizes: 156420 / 102852
Output sizes: ratio 60 ; rare 69315 ; only a 87045 ; only b 33477

only_a [('magers', 108), ('webmaster', 106), ('nareau', 105), ('guideslines', 104), ('jbl', 68), ('druxman', 61), ('salutations', 56), ('stane', 54), ('obadiah', 53), ('dessay', 52)]

ratio_a [('highly', 6.68), ('excellent', 6.51), ('wonderful', 6.1), ('loved', 4.13), ('season', 4.05), ('great', 3.79), ('fun', 3.59), ('classic', 3.48), ('recommend', 3.28), ('enjoy', 3.06), ('love', 3.0), ('best', 2.84), ('family', 2.72), ('series', 2.55), ('music', 2.16), ('years', 2.11), ('dvd', 2.03), ('set', 2.03), ('life', 1.85), ('world', 1.85), ('worth', 1.8), ('times', 1.8), ('watched', 1.79), ('fan', 1.78), ('new', 1.75), ('old', 1.67), ('video', 1.65), ('good', 1.59), ('makes', 1.59), ('watch', 1.53), ('work', 1.48), ('lot', 1.48), ('man', 1.44), ('long', 1.4), ('story', 1.39), ('little', 1.39), ('time', 1.34), ('seen', 1.33), ('funny', 1.33), ('watching', 1.3), ('real', 1.29), ('films', 1.25), ('wa

In [16]:
filter_tokens_a = filter_tokens_by_ratio(ratio_a, 1.25, 10,
['highly', 'season', 'collection', 'episodes', 'features', 'classic', 'episode', 'definitely', 'shows', 'family', 'performances', 'gives', 'young', 'series',
 'music', 'performance', 'live', 'years', 'job', 'john', 'set', 'true', 'finally', 'truly', 'world', 'different', 'dvd', 'especially', 'history',
 'life', 'cast', 'worth', 'fan', 'times', 'new', 'old', 'makes', 'watch',
 'work', 'man', 'video', 'long', 'story', 'lot', 'little', 'come', 'seen', 'watched'
])
print(filter_tokens_a)

filter_tokens_b = filter_tokens_by_ratio(ratio_b, 1.25, 10, [])
print(filter_tokens_b)

['excellent', 'wonderful', 'loved', 'great', 'fun', 'recommend', 'enjoy', 'love', 'best', 'good']
['bad', 'acting']


#### Create new matrix only consisting of filtered tokens.

In [17]:
# Remove everything but filtered tokens
matrix_filtered       = matrix.filter_tokens(filter_tokens_a)
matrix_filtered_neg_a = matrix_filtered.filter_keep_reviews(ids_neg_a)
matrix_filtered_neg_b = matrix_filtered.filter_keep_reviews(ids_neg_b)
matrix_filtered_pos_b = matrix_filtered.filter_keep_reviews(ids_pos_b)

Created: Matrix (1584098, 10), 1584098 reviews, 10 tokens
Filtering. Based on 36185 review IDs
Created: Matrix (36185, 10), 36185 reviews, 10 tokens
Filtering. Based on 21078 review IDs
Created: Matrix (21078, 10), 21078 reviews, 10 tokens
Filtering. Based on 141293 review IDs
Created: Matrix (141293, 10), 141293 reviews, 10 tokens


In [18]:
# Filter: Only texts with minimal specified positive words
reviews_neg_a = {}
for review_item in matrix_filtered_neg_a.count_token_occurences(ids_neg_a, filter_tokens_a).items():
    if review_item[1] <= 1:
        reviews_neg_a[review_item[0]] = review_item[1]
print(len(reviews_neg_a))

27668


In [19]:
# Filter: Only texts with minimal specified positive words
reviews_neg_b = {}
for review_item in matrix_filtered_neg_b.count_token_occurences(ids_neg_b, filter_tokens_a).items():
    if review_item[1] <= 1:
        reviews_neg_b[review_item[0]] = review_item[1]
print(len(reviews_neg_b))

16446


In [20]:
# Filter: Only texts with at least 5 specified positive words
reviews_pos_b = {}
for review_item in matrix_filtered_pos_b.count_token_occurences(ids_pos_b, filter_tokens_a).items():
    if review_item[1] >= 5:
        reviews_pos_b[review_item[0]] = review_item[1]
print(len(reviews_pos_b))

2888


In [21]:
# Sort by number of occurences of specified positive words
reviews_neg_a_valuesorted = {k: v for k, v in sorted(reviews_neg_a.items(), key=lambda item: item[1], reverse=True)}

end_id = 1010
print(list(reviews_neg_a_valuesorted.values())[end_id-10:end_id])
end_id = 10
print(list(reviews_neg_a_valuesorted.values())[end_id-10:end_id])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [22]:
# Sort by number of occurences of specified positive words
reviews_neg_b_valuesorted = {k: v for k, v in sorted(reviews_neg_b.items(), key=lambda item: item[1], reverse=True)}

end_id = 1010
print(list(reviews_neg_b_valuesorted.values())[end_id-10:end_id])
end_id = 10
print(list(reviews_neg_b_valuesorted.values())[end_id-10:end_id])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [23]:
# Sort by number of occurences of specified positive words
reviews_pos_b_valuesorted = {k: v for k, v in sorted(reviews_pos_b.items(), key=lambda item: item[1], reverse=True)}

end_id = 1005
print(list(reviews_pos_b_valuesorted.values())[end_id-10:end_id])
end_id = 10
print(list(reviews_pos_b_valuesorted.values())[end_id-10:end_id])

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[9, 9, 8, 8, 8, 8, 8, 8, 8, 8]


In [24]:
review_ids_neg_a = list(reviews_neg_a_valuesorted.keys())[:10000]
review_ids_neg_b = list(reviews_neg_b_valuesorted.keys())[:9000]
review_ids_pos_b = list(reviews_pos_b_valuesorted.keys())[:1000]
review_ids_b = review_ids_neg_b + review_ids_pos_b
review_ids_neg_a.sort()
review_ids_neg_b.sort()
review_ids_pos_b.sort()
review_ids_b.sort()
print(len(review_ids_neg_a), len(review_ids_neg_b), len(review_ids_pos_b), len(review_ids_b))

10000 9000 1000 10000


# Check resulting dataset

In [25]:
# A vs B
check_ratio_a, check_ratio_b, check_rare, check_only_a, check_only_b = get_ratio(
    matrix.filter_keep_reviews(review_ids_neg_a),
    matrix.filter_keep_reviews(review_ids_b),
    min_token_occurences=500)
print()
print('only_a', list(check_only_a.items())[:10])
print()
print('ratio_a', list(check_ratio_a.items())[:50])
print()
print('ratio_b', list(check_ratio_b.items())[:50])
print()
print('only_b', list(check_only_b.items())[:10])
print()
print('rare', list(check_rare.items())[:10])

Filtering. Based on 10000 review IDs
Created: Matrix (10000, 607181), 10000 reviews, 607181 tokens
Filtering. Based on 10000 review IDs
Created: Matrix (10000, 607181), 10000 reviews, 607181 tokens
Input sizes: 44124 / 48459
Output sizes: ratio 174 ; rare 27863 ; only a 16087 ; only b 20422

only_a [('pornographic', 11), ('paula', 10), ('jms', 10), ('romania', 9), ('wack', 9), ('ellroy', 9), ('stained', 8), ('idiocracy', 8), ('unentertaining', 8), ('scathing', 8)]

ratio_a [('worst', 1.54), ('waste', 1.47), ('awful', 1.47), ('stupid', 1.43), ('terrible', 1.4), ('money', 1.35), ('poor', 1.32), ('horrible', 1.32), ('thing', 1.31), ('worse', 1.3), ('bad', 1.27), ('boring', 1.25), ('horror', 1.25), ('good', 1.24), ('idea', 1.24), ('let', 1.22), ('director', 1.22), ('buy', 1.21), ('instead', 1.21), ('film', 1.19), ('care', 1.19), ('movie', 1.18), ('people', 1.18), ('acting', 1.18), ('half', 1.18), ('movies', 1.17), ('want', 1.17), ('plot', 1.16), ('saw', 1.16), ('script', 1.16), ('review', 

In [26]:
# Neg vs. Pos
check_ratio_a, check_ratio_b, check_rare, check_only_a, check_only_b = get_ratio(
    matrix.filter_keep_reviews(review_ids_neg_a + review_ids_neg_b),
    matrix.filter_keep_reviews(review_ids_pos_b),
    min_token_occurences=500)
print()
print('only_a', list(check_only_a.items())[:10])
print()
print('ratio_a', list(check_ratio_a.items())[:50])
print()
print('ratio_b', list(check_ratio_b.items())[:50])
print()
print('only_b', list(check_only_b.items())[:10])
print()
print('rare', list(check_rare.items())[:10])

Filtering. Based on 19000 review IDs
Created: Matrix (19000, 607181), 19000 reviews, 607181 tokens
Filtering. Based on 1000 review IDs
Created: Matrix (1000, 607181), 1000 reviews, 607181 tokens
Input sizes: 59005 / 25386
Output sizes: ratio 343 ; rare 19502 ; only a 39160 ; only b 5541

only_a [('redeeming', 262), ('pretentious', 155), ('amateurish', 142), ('uninteresting', 141), ('unfunny', 133), ('idiotic', 127), ('stinks', 118), ('bin', 106), ('rubbish', 103), ('ugh', 94)]

ratio_a [('waste', 80.12), ('garbage', 45.9), ('wasted', 40.5), ('worst', 29.65), ('awful', 25.89), ('lame', 24.64), ('horrible', 22.7), ('boring', 20.31), ('disappointment', 20.31), ('disappointing', 18.04), ('poorly', 17.61), ('rent', 17.49), ('terrible', 16.08), ('stupid', 13.84), ('worse', 13.7), ('money', 13.14), ('sorry', 12.5), ('ridiculous', 12.44), ('predictable', 12.2), ('disappointed', 12.01), ('cheap', 11.94), ('mess', 11.89), ('poor', 11.88), ('movie', 11.15), ('avoid', 10.87), ('bad', 10.85), ('sup

# Write results

In [27]:
def write_ids(file_path, ids_list):
    print('Writing ' + str(len(ids_list)) + ' items to ' + file_path)
    with open(file_path, 'w') as fp:
        fp.write("\n".join(str(item) for item in ids_list))

if False:
    # Note: Formerly amore-2-a-neg.txt
    write_ids(file_storage.get_storage_directory() + '/benchmark-ids/amore-2-a.txt', review_ids_neg_a)
    # Note: Deleted as included in amore-2-b.txt
    # write_ids(file_storage.get_storage_directory() + '/benchmark-ids/amore-2-b-neg.txt', review_ids_neg_b)
    # Note: Formerly amore-2-b-pos.txt and wrongly named amore-2-a-pos.txt
    write_ids(file_storage.get_storage_directory() + '/benchmark-ids/amore-2-drift.txt', review_ids_pos_b)
    write_ids(file_storage.get_storage_directory() + '/benchmark-ids/amore-2-b.txt', review_ids_b)
    write_ids(file_storage.get_storage_directory() + '/benchmark-ids/amore-2-tokens.txt', filter_tokens_a)

# Writing 10000 items to /home/eml4u/EML4U/notebooks/amore/data/benchmark-ids/amore-2-a-neg.txt
# Writing 9000 items to /home/eml4u/EML4U/notebooks/amore/data/benchmark-ids/amore-2-b-neg.txt
# Writing 1000 items to /home/eml4u/EML4U/notebooks/amore/data/benchmark-ids/amore-2-a-pos.txt
# Writing 10000 items to /home/eml4u/EML4U/notebooks/amore/data/benchmark-ids/amore-2-b.txt
# Writing 10 items to /home/eml4u/EML4U/notebooks/amore/data/benchmark-ids/amore-2-tokens.txt

# General tests

In [28]:
# Check dict review-id to number of occurences
if False:
    print('a', list(reviews_neg_b_valuesorted.keys())[0:9])
    #print('b', list(reviews_neg_b_valuesorted)[0:9])
    print(get_text(1172))

In [29]:
# Tests for matrix class

def print_matrix_info(m, t=8):
    print('Review IDs ', list(m.reviewids_to_reviewindices.keys()))
    print('Token IDs', t,  list(m.token_to_tokenindices.keys())[:t])

def get_test_matrix(print_info=False):
    r = list(vecid_revno.values())[:8]
    t = list(tokens)
    c = doc_term_matrix[:8]
    if print_info:
        print('r    ', r)
        print('t[:8]', t[:8])
    m = Matrix(c, r, t)
    return m

if False:
    # Test matrix generation
    m = get_test_matrix(True)
    print_matrix_info(m)
    m = None

if False:
    # Test filter_remove_reviews()
    m = get_test_matrix()
    revs = list(m.reviewids_to_reviewindices.keys())
    rem = revs.copy()
    print_matrix_info(m)
    print(m.get_token_data(revs[0]))
    print(m.get_token_data(revs[7]))
    print()
    rem.pop(7)
    rem.pop(5)
    rem.pop(0)
    print('Revs to remove', rem)
    m = m.filter_remove_reviews(rem)
    print_matrix_info(m)
    print(m.get_token_data(revs[0]))
    print(m.get_token_data(revs[7]))
    m = None
    revs = None
    rem = None

if False:
    # Test filter_keep_reviews()
    m = get_test_matrix()
    revs = list(m.reviewids_to_reviewindices.keys())
    print_matrix_info(m)
    print('1', m.get_token_data(revs[1]))
    print('6', m.get_token_data(revs[6]))
    print()
    revs.pop(7)
    revs.pop(5)
    revs.pop(0)
    print('Revs to keep', revs)
    m = m.filter_keep_reviews(revs)
    print_matrix_info(m)
    print('0', m.get_token_data(revs[0]))
    print('4', m.get_token_data(revs[4]))
    m = None
    revs = None
    
print_matrix_info = None
get_test_matrix = None